In [1]:
import sklearn  # collection of machine learning algorithms
from IPython import display  # pretty printing of dataframes in Jupyter notebook
# collection of functions for scientific computing and advance mathematics
import scipy as sp
import numpy as np  # foundational package for scientific computing
# collection of functions for scientific and publication-ready visualization
import matplotlib
import pandas as pd  # collection of functions for data processing and analysis modeled after R dataframes with SQL like features
import pandas as pd
import sweetviz as sv
import seaborn as sns
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

import warnings
import IPython
import sys  # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))

print("SciPy version: {}". format(sp.__version__))

print("IPython version: {}". format(IPython.__version__))

print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries


#ignore warnings
warnings.filterwarnings('ignore')
print('-'*25)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#misc libraries


#Common Model Helpers

#Visualization

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline


Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
SciPy version: 1.7.1
IPython version: 7.26.0
scikit-learn version: 0.24.2
-------------------------


In [14]:
#pathからcsvファイルを読み込む場合
data_raw = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")
df_train = data_raw.copy(deep=True)
data_cleaner = [df_train, df_test]
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print("--------------------")
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print("--------------------")
df_train.head()


Shape of Train Data
[row :891,column :12]
--------------------
Shape of Test Data
[row :418,column :11]
--------------------


,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
for dataset in data_cleaner:
    #Discrete variables
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1  # initialize to yes/1 is alone
    # now update to no/0 if family size is greater than 1
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]

    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)


In [16]:
label = LabelEncoder()
for dataset in data_cleaner:
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])

In [17]:
label = LabelEncoder()
for dataset in data_cleaner:
    # for i in df_train["Title_Code"].unique():
    #     dataset['Age'][dataset['Title_Code'] == i].fillna(
    #         dataset['Age'][df_train["Title_Code"] == i].mean(), inplace=True)
    dataset["Age"].fillna(dataset["Age"].mean(),inplace=True)
    dataset["Fare"].fillna(dataset["Fare"].mean(), inplace=True)
    dataset['AgeBin'] = pd.qcut(dataset['Age'], 3)
    dataset['Age_Code'] = label.fit_transform(dataset['AgeBin'])


In [18]:
df_train = pd.get_dummies(df_train, columns=["Embarked"])
df_test = pd.get_dummies(df_test, columns=["Embarked"])
df_train.head()

,PassengerId,Perished,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,FareBin,Sex_Code,Embarked_Code,Title_Code,FareBin_Code,AgeBin,Age_Code,Embarked_C,Embarked_Q,Embarked_S
0,1,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,"(-0.001, 7.91]",1,2,11,0,"(0.419, 25.0]",0,0,0,1
1,2,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,"(31.0, 512.329]",0,0,12,3,"(31.0, 80.0]",2,1,0,0
2,3,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,"(7.91, 14.454]",0,2,8,1,"(25.0, 31.0]",1,0,0,1
3,4,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,"(31.0, 512.329]",0,2,12,3,"(31.0, 80.0]",2,0,0,1
4,5,1,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,"(7.91, 14.454]",1,2,11,1,"(31.0, 80.0]",2,0,0,1


In [ ]:
#save csv
import sys
import os
PATH_dir = "./df"
df_train.to_csv(os.path.join(
    PATH_dir, "EDA_09_MLP_train.csv"), index_label=False)
df_test.to_csv(os.path.join(
    PATH_dir, "EDA_09_MLP_test.csv"), index_label=False)
